# LangChain: Memory

## Disclaimer

This example is modified version of a jupyter notebook associate to the course [https://learn.deeplearning.ai/langchain/lesson/1/introduction](https://learn.deeplearning.ai/langchain/lesson/1/introduction).

In this version I adapted to run with Azure OpenaiKey, so I've made really small modidfication, I suggest you to do the original course.


## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [3]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"

# Remember that you need to set the OPENAI_API_BASE to point openai to your specific deployment.

openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# print base to verify you are pointint to the right deployment
print(openai.api_base)

https://alkopenai.openai.azure.com/


In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


This is a code that create Azure chat openai, you need to specify deployment_name, remember to specify openai version **or you will get an error**. 

Then the special part is the memory, that allows the llm object to have actual memory on the current conversation.

In [6]:
llm = AzureChatOpenAI(
    temperature=0.0, 
    deployment_name="gpt35", 
    openai_api_version="2023-03-15-preview")
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [7]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [8]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'The answer to 1+1 is 2.'

In [9]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew, as you mentioned earlier.'

Print the buffer, just to know what is inside memory and what is passed from the various call

In [10]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Andrew, as you mentioned earlier.


Use pretthy print to print memory variables to understand how everything is kept into memory.

In [12]:
from pprint import pprint
memory_variables = memory.load_memory_variables({})
pprint(memory_variables)

{'history': 'Human: Hi, my name is Andrew\n'
            "AI: Hello Andrew, it's nice to meet you. My name is AI. How can I "
            'assist you today?\n'
            'Human: What is 1+1?\n'
            'AI: The answer to 1+1 is 2.\n'
            'Human: What is my name?\n'
            'AI: Your name is Andrew, as you mentioned earlier.'}


In [13]:
# reset the value of the memory buffer creating a new memory object
memory = ConversationBufferMemory()

In [14]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [15]:
pprint(memory.buffer)

"Human: Hi\nAI: What's up"


In [17]:
pprint(memory.load_memory_variables({}))

{'history': "Human: Hi\nAI: What's up"}


In [18]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [19]:
pprint(memory.load_memory_variables({}))

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}


## ConversationBufferWindowMemory

In [20]:
from langchain.memory import ConversationBufferWindowMemory

In [21]:
memory = ConversationBufferWindowMemory(k=1)               

In [22]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


You will verify with the following code that since we have a k=1 only latest piece of the conversation is remembered

In [23]:
pprint(memory.load_memory_variables({}))

{'history': 'Human: Not much, just hanging\nAI: Cool'}


In [26]:
# llm = ChatOpenAI(temperature=0.0)
llm = AzureChatOpenAI(
    temperature=0.0, 
    deployment_name="gpt35", 
    openai_api_version="2023-03-15-preview")
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [27]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

In [28]:
conversation.predict(input="What is 1+1?")

'The answer to 1+1 is 2.'

In [29]:
conversation.predict(input="What is my name?")

"I'm sorry, I don't have access to that information. Could you please tell me your name?"

## ConversationTokenBufferMemory

In [ ]:
#!pip install tiktoken

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [ ]:
memory.load_memory_variables({})

## ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory


In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")

In [ ]:
memory.load_memory_variables({})